## 正規表達法斷句

In [1]:
import re
article = '''

【綜合報導】違建真害命！違建大火連續燒，近2周內，光雙北市至少有10人因密集隔間違建火警喪命，新北市府日前展開大動作，強拆一棟2層樓加蓋成6層樓、隔成158間房分租的離譜違建，強調明起全面執行頂加分租套房拆除。台北市長柯文哲昨也表示，考慮拿掉前市長陳水扁下的「違建特赦令」，涉及公安的大型違建，不論是既存違建或是新違建，都要依法處置。目前雙北共有29萬戶列管違建，若不改善，恐將面臨拆除命運。
'''

In [4]:
re.split('！|，|。',article)

['\n\n【綜合報導】違建真害命',
 '違建大火連續燒',
 '近2周內',
 '光雙北市至少有10人因密集隔間違建火警喪命',
 '新北市府日前展開大動作',
 '強拆一棟2層樓加蓋成6層樓、隔成158間房分租的離譜違建',
 '強調明起全面執行頂加分租套房拆除',
 '台北市長柯文哲昨也表示',
 '考慮拿掉前市長陳水扁下的「違建特赦令」',
 '涉及公安的大型違建',
 '不論是既存違建或是新違建',
 '都要依法處置',
 '目前雙北共有29萬戶列管違建',
 '若不改善',
 '恐將面臨拆除命運',
 '\n']

## 使用Jieba

In [5]:
! pip install jieba

In [8]:
import jieba
for ele in jieba.cut('大巨蛋案對市府同仁下封口令? 柯P否認'):
    print(ele)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 1.176 seconds.
Prefix dict has been built succesfully.


大
巨蛋
案對
市府
同仁
下
封口令
?
 
柯
P
否認


In [10]:
'/'.join(jieba.cut('大巨蛋案對市府同仁下封口令? 柯P否認'))

'大/巨蛋/案對/市府/同仁/下/封口令/?/ /柯/P/否認'

In [12]:
'/'.join(jieba.cut('大巨蛋案對市府同仁下封口令? 柯P否認', cut_all=True))

'大/巨蛋/案/對/市府/同仁/下/封口/封口令/口令////柯/P/否/認'

In [13]:
'/'.join(jieba.cut('大巨蛋案對市府同仁下封口令? 柯P否認'))

'大/巨蛋/案對/市府/同仁/下/封口令/?/ /柯/P/否認'

## 讀取使用者字典

In [18]:
jieba.load_userdict('userdict.txt')

In [19]:
'/'.join(jieba.cut('大巨蛋案對市府同仁下封口令? 柯P否認'))

'大巨蛋/案對/市府/同仁/下/封口令/?/ /柯P/否認'

## 抓出詞性

In [20]:
import jieba.posseg as pseg
words = pseg.cut('大巨蛋案對市府同仁下封口令? 柯P否認')
for w in words:
    print(w.word, w.flag)

大巨蛋 n
案 ng
對 p
市府 n
同仁 nr
下 f
封口令 n
? x
  x
柯P n
否認 v


In [21]:
sentence = '大巨蛋案對市府同仁下封口令? 柯P否認'
for tw in jieba.tokenize(sentence):
    print(tw)

('大巨蛋', 0, 3)
('案對', 3, 5)
('市府', 5, 7)
('同仁', 7, 9)
('下', 9, 10)
('封口令', 10, 13)
('?', 13, 14)
(' ', 14, 15)
('柯P', 15, 17)
('否認', 17, 19)


In [23]:
import jieba.analyse
tags = jieba.analyse.extract_tags(sentence, 1)
print(tags)

['封口令']


In [24]:
import jieba.analyse
tags = jieba.analyse.extract_tags(sentence, 1, allowPOS=['nr'])
print(tags)

['同仁']


## 擴充字典
- https://www.moedict.tw/

- https://zh.wikipedia.org/wiki/%E5%94%90%E7%B4%8D%E5%BE%B7%C2%B7%E5%B7%9D%E6%99%AE

## 使用新聞關鍵字

In [26]:
import requests
from bs4 import BeautifulSoup
res = requests.get('http://news.ltn.com.tw/news/business/breakingnews/2272811')
soup = BeautifulSoup(res.text, 'html.parser')

In [29]:
keywords = [ele.text for ele in soup.select('.keyword a')]

In [31]:
with open('userdict.txt', 'a', encoding='utf-8') as f:
    f.write('\n')
    for keyword in keywords:
        f.write(keyword + '\n')

## 取得保險詞彙

In [37]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.ib.gov.tw/ch/home.jsp?id=59&parentpath=0,6&mcustomize='
payload = {
'id':'59',
'contentid':'59',
'parentpath':'0,6',
'mcustomize':'bilingual_list.jsp',
'ckeyword':'請輸入中文關鍵字',
'ekeyword':'請輸入英文關鍵字',
'page':'3'
}

res = requests.post(url, data = payload)

soup = BeautifulSoup(res.text, 'html.parser')
keywords = [ele.text for ele in soup.select('.bich_name_con')]
with open('userdict.txt', 'a', encoding='utf-8') as f:
    f.write('\n')
    for keyword in keywords:
        f.write(keyword + '\n')

## 將PDF 轉成文字

In [38]:
! pip install pdfminer3k

In [40]:
res = requests.get('https://www.fubon.com/life/public_info/public_info_04/5-12.pdf')
with open('fubon.pdf', 'wb') as f:
    f.write(res.content)

In [ ]:
## from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine

s = ''
fp = open('fubon.pdf', 'rb')
parser = PDFParser(fp)
doc = PDFDocument()
parser.set_document(doc)
doc.set_parser(parser)
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            #print(lt_obj.get_text())
            pass

## 將文字轉成 Word 檔案

In [44]:
! pip install python-docx

  Running setup.py bdist_wheel for python-docx: started
  Running setup.py bdist_wheel for python-docx: finished with status 'done'
  Stored in directory: C:\Users\User\AppData\Local\pip\Cache\wheels\cc\74\10\42b00d7d6a64cf21f194bfef9b94150009ada880f13c5b2ad3
Successfully built python-docx


In [45]:
from docx import Document
from docx.shared import Inches

document = Document()

document.add_heading('Document Title', 0)

p = document.add_paragraph('A plain paragraph having some ')
p.add_run('bold').bold = True
p.add_run(' and some ')
p.add_run('italic.').italic = True

document.add_heading('Heading, level 1', level=1)
document.add_paragraph('Intense quote', style='IntenseQuote')

document.add_paragraph(
    'first item in unordered list', style='ListBullet'
)
document.add_paragraph(
    'first item in ordered list', style='ListNumber'
)


document.add_page_break()

document.save('demo.docx')

C:\ProgramData\Anaconda3\lib\site-packages\docx\styles\styles.py:54: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  warn(msg, UserWarning)


## 將PDF轉 WORD

In [46]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
from docx import Document
from docx.shared import Inches

document = Document()


fp = open('fubon.pdf', 'rb')
parser = PDFParser(fp)
doc = PDFDocument()
parser.set_document(doc)
doc.set_parser(parser)
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            document.add_paragraph(lt_obj.get_text())
    document.add_page_break()
document.save('fubon.docx')

## NGRAM

In [51]:
sentence = '那我們酸民婉君也可以報名嗎'

In [52]:
sentence[0:2]

'那我'

In [53]:
sentence[1:3]

'我們'

In [54]:
for i in range(0, len(sentence) - 2 + 1):
    print(sentence[i:i+2])

那我
我們
們酸
酸民
民婉
婉君
君也
也可
可以
以報
報名
名嗎


In [56]:
for i in range(0, len(sentence) - 3 + 1):
    print(sentence[i:i+3])

那我們
我們酸
們酸民
酸民婉
民婉君
婉君也
君也可
也可以
可以報
以報名
報名嗎


In [60]:
def ngram(sentence, n = 2):
    words = []
    for i in range(0, len(sentence) - n + 1):
        words.append(sentence[i:i+n])
    return words

In [61]:
ngram(sentence, 4)

['那我們酸',
 '我們酸民',
 '們酸民婉',
 '酸民婉君',
 '民婉君也',
 '婉君也可',
 '君也可以',
 '也可以報',
 '可以報名',
 '以報名嗎']

In [62]:
news = '''
全台最後一場週年慶今日由新光三越台北站前店壓軸登場，化妝品推出20萬組明星特惠組、保暖外套全面5折起，吸引上千民眾排隊搶購，館內一度塞車7分鐘。周末又有一波強烈冷氣團來襲，百貨業者看好禦寒商品、抗空污家電持續成長，預估首日來客數可達7萬人，業績上看4.5億元，成長1%。
 
台北站前店店長曹龍生表示，今年化妝品表現最好，各品牌口紅都熱賣，年輕美眉也人手一支，為彩妝引進許多新客，帶動業績大幅成長。另外近來全台深受空污影響，讓空氣清淨機熱賣，如LG空氣清淨機大缺貨，家電新機種多優先在百貨開賣，搭配滿千送百更划算。
 
冷氣團一波接著一波，毛料外套、羽絨被也相當受歡迎，如羽毛工房限量100件，開店不久即搶光。新光三越台北站前店推出話題大衣全面5折起，共計200櫃全面力拚比市場再低1折，500多件保暖外套平均1000~1500元；保暖冬被1折起。營業副理陳英玒看好在空污議題、低溫助攻下，家電業績可成長30%。
 
民眾陳小姐表示今天以靴款為採買重點；從事行政的王小姐則主攻保養品、彩妝、香水，偏好在同一櫃買齊，折扣累積更划算，預算無上限；從事補教業的蒲先生同樣以購買保養品為主，保濕、眼霜、精華液等，大約花費1萬元。
 
新光三越台北站前店今起至24日為期18天，首波強勢祭出超值禮券回饋，20日前化妝品滿2,000送200、全館滿5,000送500、大家電/法雅客滿萬送500、13日前加碼貴賓卡獨享大家電滿萬送仟。
'''

In [67]:
from collections import Counter
words = ngram(news, n = 2)
words_dic = Counter(words)
for k , v in words_dic.most_common(30):
    if v >= 3:
        print(k, v)

00 13
家電 5
。
 5
台北 4
北站 4
站前 4
前店 4
20 4
成長 4

  4
 
 4
50 4
新光 3
光三 3
三越 3
越台 3
化妝 3
妝品 3
保暖 3
外套 3
全面 3
折起 3
一波 3
空污 3
業績 3
賣， 3
0、 3


In [69]:
from collections import Counter
words = ngram(news, n = 3)
words_dic = Counter(words)
for k , v in words_dic.most_common(30):
    if v >= 3:
        print(k, v)

台北站 4
北站前 4
站前店 4
。
  4

 
 4
500 4
新光三 3
光三越 3
三越台 3
越台北 3
化妝品 3
000 3
00、 3


In [70]:
from collections import Counter
words = ngram(news, n = 4)
words_dic = Counter(words)
for k , v in words_dic.most_common(30):
    if v >= 3:
        print(k, v)

台北站前 4
北站前店 4
。
 
 4
新光三越 3
光三越台 3
三越台北 3
越台北站 3


In [78]:
import re
for ele in re.split('，|。|、|；|/', news):
    print(ele)


全台最後一場週年慶今日由新光三越台北站前店壓軸登場
化妝品推出20萬組明星特惠組
保暖外套全面5折起
吸引上千民眾排隊搶購
館內一度塞車7分鐘
周末又有一波強烈冷氣團來襲
百貨業者看好禦寒商品
抗空污家電持續成長
預估首日來客數可達7萬人
業績上看4.5億元
成長1%

 
台北站前店店長曹龍生表示
今年化妝品表現最好
各品牌口紅都熱賣
年輕美眉也人手一支
為彩妝引進許多新客
帶動業績大幅成長
另外近來全台深受空污影響
讓空氣清淨機熱賣
如LG空氣清淨機大缺貨
家電新機種多優先在百貨開賣
搭配滿千送百更划算

 
冷氣團一波接著一波
毛料外套
羽絨被也相當受歡迎
如羽毛工房限量100件
開店不久即搶光
新光三越台北站前店推出話題大衣全面5折起
共計200櫃全面力拚比市場再低1折
500多件保暖外套平均1000~1500元
保暖冬被1折起
營業副理陳英玒看好在空污議題
低溫助攻下
家電業績可成長30%

 
民眾陳小姐表示今天以靴款為採買重點
從事行政的王小姐則主攻保養品
彩妝
香水
偏好在同一櫃買齊
折扣累積更划算
預算無上限
從事補教業的蒲先生同樣以購買保養品為主
保濕
眼霜
精華液等
大約花費1萬元

 
新光三越台北站前店今起至24日為期18天
首波強勢祭出超值禮券回饋
20日前化妝品滿2,000送200
全館滿5,000送500
大家電
法雅客滿萬送500
13日前加碼貴賓卡獨享大家電滿萬送仟




In [79]:
def removeKey(sentence, keywords):
    for keyword in keywords:
        sentence = sentence.replace(keyword, '')
    return sentence

In [82]:
def removeKey2(sentence, keywords):
    for keyword in keywords:
        sentence = ''.join(sentence.split(keyword))
    return sentence

In [83]:
removeKey(news, ['化妝品', '新光三越'])
removeKey2(news, ['化妝品', '新光三越'])

'\n全台最後一場週年慶今日由台北站前店壓軸登場，推出20萬組明星特惠組、保暖外套全面5折起，吸引上千民眾排隊搶購，館內一度塞車7分鐘。周末又有一波強烈冷氣團來襲，百貨業者看好禦寒商品、抗空污家電持續成長，預估首日來客數可達7萬人，業績上看4.5億元，成長1%。\n \n台北站前店店長曹龍生表示，今年表現最好，各品牌口紅都熱賣，年輕美眉也人手一支，為彩妝引進許多新客，帶動業績大幅成長。另外近來全台深受空污影響，讓空氣清淨機熱賣，如LG空氣清淨機大缺貨，家電新機種多優先在百貨開賣，搭配滿千送百更划算。\n \n冷氣團一波接著一波，毛料外套、羽絨被也相當受歡迎，如羽毛工房限量100件，開店不久即搶光。台北站前店推出話題大衣全面5折起，共計200櫃全面力拚比市場再低1折，500多件保暖外套平均1000~1500元；保暖冬被1折起。營業副理陳英玒看好在空污議題、低溫助攻下，家電業績可成長30%。\n \n民眾陳小姐表示今天以靴款為採買重點；從事行政的王小姐則主攻保養品、彩妝、香水，偏好在同一櫃買齊，折扣累積更划算，預算無上限；從事補教業的蒲先生同樣以購買保養品為主，保濕、眼霜、精華液等，大約花費1萬元。\n \n台北站前店今起至24日為期18天，首波強勢祭出超值禮券回饋，20日前滿2,000送200、全館滿5,000送500、大家電/法雅客滿萬送500、13日前加碼貴賓卡獨享大家電滿萬送仟。\n'

In [86]:
import re
for ele in re.split('，|。|、|；|/', news):
    print(ele)


全台最後一場週年慶今日由新光三越台北站前店壓軸登場
化妝品推出20萬組明星特惠組
保暖外套全面5折起
吸引上千民眾排隊搶購
館內一度塞車7分鐘
周末又有一波強烈冷氣團來襲
百貨業者看好禦寒商品
抗空污家電持續成長
預估首日來客數可達7萬人
業績上看4.5億元
成長1%

 
台北站前店店長曹龍生表示
今年化妝品表現最好
各品牌口紅都熱賣
年輕美眉也人手一支
為彩妝引進許多新客
帶動業績大幅成長
另外近來全台深受空污影響
讓空氣清淨機熱賣
如LG空氣清淨機大缺貨
家電新機種多優先在百貨開賣
搭配滿千送百更划算

 
冷氣團一波接著一波
毛料外套
羽絨被也相當受歡迎
如羽毛工房限量100件
開店不久即搶光
新光三越台北站前店推出話題大衣全面5折起
共計200櫃全面力拚比市場再低1折
500多件保暖外套平均1000~1500元
保暖冬被1折起
營業副理陳英玒看好在空污議題
低溫助攻下
家電業績可成長30%

 
民眾陳小姐表示今天以靴款為採買重點
從事行政的王小姐則主攻保養品
彩妝
香水
偏好在同一櫃買齊
折扣累積更划算
預算無上限
從事補教業的蒲先生同樣以購買保養品為主
保濕
眼霜
精華液等
大約花費1萬元

 
新光三越台北站前店今起至24日為期18天
首波強勢祭出超值禮券回饋
20日前化妝品滿2,000送200
全館滿5,000送500
大家電
法雅客滿萬送500
13日前加碼貴賓卡獨享大家電滿萬送仟




In [88]:
import re
sentenceAry = []
for ele in re.split('，|。|、|；|/', news):
    sentenceAry.append(ele)

In [112]:
news = '''
行政院院會今通過《礦業法》修正草案，在礦場補做環評方面，採面積及產量分級管制，礦業用地面積大於2公頃、最近5年內年平均生產量5萬公噸以上的大型礦場，3年內補做環評；在此標準以下的小型礦場，則必須在5年內辦理環境影響調查、分析，並提出因應對策，若屆時未依法辦理補做環評，主管機關可處罰並要求限期改善，若屆期仍未改善者，最重可廢止其原核定的礦業用地。
 
經濟部礦物局長徐景文今在院會後記者會表示，若修法通過，大型礦場3年內依《環境影響評估法》補做環評，共有6個水泥礦場，包括榮豐、台塑、潤泰2個、宜大、亞泥；小型礦場共有60個須在5年內依《環評法》辦理環境影響調查、分析。 

經濟部礦業局官員在院會後記者會則表示，目前約有50件已申請礦展限的案件送到礦務局辦理展現，但立法院日前決議在明年3月前不得准駁展限。根據先前聽取學者專家意見，未來《礦業法》修法通過後，該50件已申請礦展限的案件將依據《中央標準法》的相關規定，針對法規採取「從優從新」方向辦理，因此須補辦環評、但不用經原住民族諮商同意。
 
根據《礦業法》修正草案，「修正之條文施行後提出之礦業權展限申請時，其原核定礦業用地位於原住民族土地或部落及其周邊一定範圍內之公有土地，礦業權者應依《原住民族基本法》第21條(原住民族的諮商同意權)辦理。礦業局官員說，因此該50件是在修正之條文施行「前」提出，因此依法就無須經《原基本》第21條(原住民族的諮商同意權)辦理。
 
據該草案規定，凡未曾實施環境影響評估者，在礦場補做環評方面，採面積及產量分級管制，礦業用地面積大於2公頃、最近5年內年平均生產量5萬公噸以上的大型礦場，3年內補做環評；在此標準以下的小型礦場，則必須在5年內辦理環境影響調查、分析。

'''

In [119]:
import re
sentenceAry = []
for ele in re.split('，|。|、|；|/|》|《|\)|「|\(|」', news):
    sentenceAry.append(ele)
sentenceAry

['\n行政院院會今通過',
 '礦業法',
 '修正草案',
 '在礦場補做環評方面',
 '採面積及產量分級管制',
 '礦業用地面積大於2公頃',
 '最近5年內年平均生產量5萬公噸以上的大型礦場',
 '3年內補做環評',
 '在此標準以下的小型礦場',
 '則必須在5年內辦理環境影響調查',
 '分析',
 '並提出因應對策',
 '若屆時未依法辦理補做環評',
 '主管機關可處罰並要求限期改善',
 '若屆期仍未改善者',
 '最重可廢止其原核定的礦業用地',
 '\n \n經濟部礦物局長徐景文今在院會後記者會表示',
 '若修法通過',
 '大型礦場3年內依',
 '環境影響評估法',
 '補做環評',
 '共有6個水泥礦場',
 '包括榮豐',
 '台塑',
 '潤泰2個',
 '宜大',
 '亞泥',
 '小型礦場共有60個須在5年內依',
 '環評法',
 '辦理環境影響調查',
 '分析',
 ' \n\n經濟部礦業局官員在院會後記者會則表示',
 '目前約有50件已申請礦展限的案件送到礦務局辦理展現',
 '但立法院日前決議在明年3月前不得准駁展限',
 '根據先前聽取學者專家意見',
 '未來',
 '礦業法',
 '修法通過後',
 '該50件已申請礦展限的案件將依據',
 '中央標準法',
 '的相關規定',
 '針對法規採取',
 '從優從新',
 '方向辦理',
 '因此須補辦環評',
 '但不用經原住民族諮商同意',
 '\n \n根據',
 '礦業法',
 '修正草案',
 '',
 '修正之條文施行後提出之礦業權展限申請時',
 '其原核定礦業用地位於原住民族土地或部落及其周邊一定範圍內之公有土地',
 '礦業權者應依',
 '原住民族基本法',
 '第21條',
 '原住民族的諮商同意權',
 '辦理',
 '礦業局官員說',
 '因此該50件是在修正之條文施行',
 '前',
 '提出',
 '因此依法就無須經',
 '原基本',
 '第21條',
 '原住民族的諮商同意權',
 '辦理',
 '\n \n據該草案規定',
 '凡未曾實施環境影響評估者',
 '在礦場補做環評方面',
 '採面積及產量分級管制',
 '礦業用地面積大於2公頃',
 '最近5年內年平均生產量5萬公噸以上的大型礦場',
 '3年內補做環評',


In [123]:
import re
keywords = []
for term_length in range(5,1,-1):
    ret = []
    for sentence in sentenceAry:
        s = removeKey(sentence, keywords)
        ngram_words = ngram(s, term_length)
        ret.extend(ngram_words)
    c = Counter(ret)
    for k, v in c.most_common():
        m = re.match('[\u4e00-\u9fa5]+', k)
        if v >= 3 and m:
            keywords.append(k)
print(keywords)

['須在5年內', '辦理環境影', '理環境影響', '環境影響調', '境影響調查', '補做環評', '原住民族', '礦業用地', '大型礦場', '小型礦場', '諮商同意', '礦業法', '響調查', '年內', '辦理', '修正', '面積', '產量', '礦業', '展限', '院會', '通過', '草案', '礦場', '標準', '分析', '提出', '申請', '因此']


## 文字雲

In [124]:
news = '''
大陸冷氣團襲台，適逢水氣充足，玉山今日凌晨二度降下瑞雪！玉山北峰氣象站工作人員觀測到零時30分起飄落雪花，約莫持續到凌晨2時，降雪達1.5公分，上午目測累積雪量達到2公分，舉目望去一片雪白景象美不勝收，宣告今年冬季正式來臨。

玉山國家公園管理處副處長林文和表示，今晨排雲山莊工作人員巡查玉山、勘查玉山主峰步道的雪況，發現從排雲山莊往上1公里步道開始有積雪現象，沿線步道積雪高度約1至2公分，由於步道濕滑，雖然排雲山莊目前積雪未達5公分，並未啟動雪季管制，但仍採柔性勸導，呼籲登山客須備齊頭盔、繩索、安全扣環、冰爪、冰斧等裝備，切勿貿然登頂。

合歡山這頭接續前兩日追雪潮，上午卻因溫度不夠低，十時許武嶺氣溫2度，現場積雪融化八成，百餘名遊客收集殘雪堆小雪人過過癮。部分民眾在上山賞雪途中意外目睹廬山、清境農場方向大片秀麗雲海，紛紛停下腳步駐足拍照，直呼「美麗的意外！」

陳姓民眾一早和妻子從桃園龜山駕車上山賞雪，見武嶺地區濃霧壟罩、殘雪消退，內心難掩失望，但在下山途中行經台14甲線28公里處，望見雲海美景立刻下車拍照，彌補缺憾。兩人緩步欣賞遠處山巒層層疊疊，雲朵如花團錦簇，讚嘆景色誘人，「有種小家碧玉的秀麗！」另有馬來西亞、新加坡等外國旅行團一行十餘遊客，儘管只見殘雪，仍大聲驚嘆，開心表示不虛此行，來台旅遊九天收穫豐碩。
'''

In [126]:
import jieba

words_ary = []
for word in jieba.cut(news):
    words_ary.append(word)

In [128]:
from collections import Counter
words_ary = [word for word in jieba.cut(news)]
c = Counter(words_ary)

In [133]:
for k, v in c.most_common(100):
    if len(k) >= 2 and v >= 2:
        print(k,v)

玉山 4
公分 4
步道 4
積雪 3
凌晨 2
工作 2
上午 2
表示 2
排雲山莊 2
公里 2
上山 2
賞雪 2
途中 2
意外 2
拍照 2


## 將量化統計包裝成函數

In [135]:
def wordCount(news):
    words_ary = [word for word in jieba.cut(news)]
    c = Counter(words_ary)
    for k, v in c.most_common(100):
        if len(k) >= 2 and v >= 2:
            print(k,v)

In [136]:
news = '''
俄羅斯遭IOC(International Olympic Committee，國際奧林匹克委員會)禁止以國家名義參加明年平昌冬季奧運，通過藥檢該國運動員只能以獨立身分、「來自俄羅斯的奧林匹克運動員」參賽。俄羅斯總統普丁(Vladimir Putin)首次公開立場，表示不會阻止運動員獨立參賽，但不無反諷指出IOC只有對俄羅斯嚴格，也否認國家主導使用禁藥。
 
普丁說：「我們不會採取任何阻擋運動員參賽措施。如果他們決定以獨立運動員身分參賽，我們當然還是會放行。我很關心這些選手，認識其中很多人，也把他們當朋友。他們一生都在為比賽不停奮鬥。」
 
但話鋒一轉，普丁開酸：「不像其他國運動員，俄羅斯選手都會接受三重檢查。這意味什麼？所以我們不會設限，讓他們參與奧運變不可能。IOC之前指控我們以國家主導禁藥計畫的調查也沒有得到結論。懲罰應該是給有罪的人，如果從來沒有國家主導或資助的禁藥系統，為什麼運動員不能揮著俄羅斯國旗參賽？」

'''

In [137]:
wordCount(news)

運動員 5
參賽 5
俄羅斯 4
我們 4
IOC 3
普丁 3
不會 3
家主 3
他們 3
國運動員 2
身分 2
獨立 2
禁藥 2
如果 2
選手 2
什麼 2


## 蘋果新聞最長出現字詞統計

In [139]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://tw.appledaily.com/new/realtime/1')
soup = BeautifulSoup(res.text, 'html.parser')
news_ary = []
for ele in soup.select('.rtddt a'):
    res2 = requests.get(ele['href'])
    soup2 = BeautifulSoup(res2.text, 'html.parser')
    content = soup2.select_one('.ndArticle_margin p').text
    news_ary.append(content)

In [141]:
words = []
for news in news_ary:
    words.extend([ele for ele in jieba.cut(news)])

In [143]:
from collections import Counter
c = Counter(words)
for k, v in c.most_common(100):
    if len(k) >= 2 and v >= 2:
        print(k,v)

法官 40
報導 36
表示 30
台灣 25
警察 23
公司 21
益生菌 17
和解 17
因為 16
社會 16
球菌 16
蘋果 16
新聞 15
部分 15
台北 15
因此 15
中心 14
12 14
美國 14
投資人 14
要求 13
今年 13
腸道 13


## TFIDF計算

In [144]:
a, abb,abc = ['a'], ['a', 'b', 'b'], ['a', 'b', 'c']

In [145]:
D = [a,abb, abc]

In [147]:
import scipy as sp

In [148]:
# tfidf('a', a, D)
tf  = 1/1
idf = sp.log(3/3)
tf * idf

0.0

In [149]:
#tfidf('a', abb, D)
tf = 1/3
idf = sp.log(3/ 3)
tf * idf 

0.0

In [151]:
#tfidf('b', abb, D)
tf = 2/3
idf = sp.log(3/2)
tf * idf

0.27031007207210955

In [152]:
# tfidf('a', abc , D)
0

0

In [153]:
#tfidf('b', abc, D)
tf = 1/3
idf = sp.log(3/2)
tf*idf

0.13515503603605478

In [154]:
# tfidf('c', abc, D)
tf = 1/3
idf = sp.log(3/1)
tf * idf

0.36620409622270322

In [155]:
def tfidf(t, d, D):
    tf  = d.count(t) / len(d)
    idf = sp.log(len(D)/len([e for e in D if t in e ]))
    return tf * idf

In [158]:
tfidf('b', abb, D)

0.27031007207210955

In [157]:
tfidf('b', abc, D)

0.13515503603605478

In [156]:
tfidf('c', abc, D)

0.36620409622270322